In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
train_data = pd.read_csv('/kaggle/input/cat-in-the-dat/train.csv', index_col = 'id')
test_data = pd.read_csv('/kaggle/input/cat-in-the-dat/test.csv', index_col= 'id')

In [3]:
train_data.info(), test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   bin_0   300000 non-null  int64 
 1   bin_1   300000 non-null  int64 
 2   bin_2   300000 non-null  int64 
 3   bin_3   300000 non-null  object
 4   bin_4   300000 non-null  object
 5   nom_0   300000 non-null  object
 6   nom_1   300000 non-null  object
 7   nom_2   300000 non-null  object
 8   nom_3   300000 non-null  object
 9   nom_4   300000 non-null  object
 10  nom_5   300000 non-null  object
 11  nom_6   300000 non-null  object
 12  nom_7   300000 non-null  object
 13  nom_8   300000 non-null  object
 14  nom_9   300000 non-null  object
 15  ord_0   300000 non-null  int64 
 16  ord_1   300000 non-null  object
 17  ord_2   300000 non-null  object
 18  ord_3   300000 non-null  object
 19  ord_4   300000 non-null  object
 20  ord_5   300000 non-null  object
 21  day     300000 non-null  int64 
 22  m

(None, None)

In [4]:
train_data.shape, test_data.shape

((300000, 24), (200000, 23))

In [ ]:
test_data.isna().sum()

In [ ]:
display(train_data.tail())
display(test_data.tail())

In [5]:
binary_features = [f'bin_{i}' for i in range(5)]
nominal_features = [f'nom_{i}' for i in range(5)]
ordinal_features = [f'ord_{i}' for i in range(6)]
binary_features, nominal_features, ordinal_features

(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4'],
 ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4'],
 ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5'])

In [ ]:
train_data.head()

In [6]:
print(set(test_data['ord_0'].unique()) == set(train_data['ord_0'].unique()))
print(set(test_data['ord_1'].unique()) == set(train_data['ord_1'].unique()))
print(set(test_data['ord_2'].unique()) == set(train_data['ord_2'].unique()))
print(set(test_data['ord_3'].unique()) == set(train_data['ord_3'].unique()))
print(set(test_data['ord_4'].unique()) == set(train_data['ord_4'].unique()))
print(set(test_data['ord_5'].unique()) == set(train_data['ord_5'].unique()))

True
True
True
True
True
True


In [7]:
print(set(test_data['nom_0'].unique()) == set(train_data['nom_0'].unique()))
print(set(test_data['nom_1'].unique()) == set(train_data['nom_1'].unique()))
print(set(test_data['nom_2'].unique()) == set(train_data['nom_2'].unique()))
print(set(test_data['nom_3'].unique()) == set(train_data['nom_3'].unique()))
print(set(test_data['nom_4'].unique()) == set(train_data['nom_4'].unique()))

True
True
True
True
True


In [ ]:
test_data['bin_3'].value_counts()

In [ ]:
test_data['bin_4'].value_counts()

In [15]:
train_data = pd.read_csv('/kaggle/input/cat-in-the-dat/train.csv', index_col = 'id')
test_data = pd.read_csv('/kaggle/input/cat-in-the-dat/test.csv', index_col= 'id')

In [16]:
def preprocess_data(data, is_train):
    temp_data = data
    temp_data = temp_data.reset_index(drop=True)
    #binary data 5개 열 처리
    bin_map_tf = {'T':1, 'F':0}
    bin_map_yn = {'Y':1, 'N':0}
    temp_data['bin_3'] = temp_data['bin_3'].map(bin_map_tf)
    temp_data['bin_4'] = temp_data['bin_4'].map(bin_map_yn)
    
    #nominal data drop할것들 drop
    drop_cols = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
    temp_data = temp_data.drop(columns=drop_cols)
    nominal_features = [f'nom_{i}' for i in range(5)]
    
    for feature in nominal_features:
        encoder = OneHotEncoder(handle_unknown='ignore')
        data_cat = encoder.fit_transform(np.array(temp_data[feature]).reshape(-1,1))
        data_cat = data_cat.toarray()
        #print(encoder.categories_)
        data_cat = pd.DataFrame(data_cat, columns=[f'{feature}_'+str(col) for col in encoder.categories_[0]])
        temp_data = temp_data.drop(columns=feature)
        temp_data = pd.concat([temp_data, data_cat], axis=1)
        
    #ordinal data 
    ord0_map = {1: 1, 2: 2, 3: 3}
    ord1_map = {'Novice': 1, 'Contributor': 2, 'Expert': 3, 'Master': 4, 'Grandmaster': 5}
    ord2_map = {'Freezing': 1, 'Cold': 2, 'Warm': 3, 'Hot': 4, 'Boiling Hot': 5, 'Lava Hot': 6}
    ord3_map = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'm': 12, 'l': 13, 'n': 14, 'o': 15}
    ord4_map = {'A': 1, 'B': 2, 'D': 3, 'E': 4, 'C': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'K': 10, 'J': 11, 'M': 12, 'L': 13, 'O': 14, 'N': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'X': 23, 'W': 24, 'Y': 25, 'Z': 26}
    ord5_map = {'Cl': 1, 'Ai': 2, 'Aj': 3, 'Kf': 4, 'CL': 5, 'Bn': 6, 'Id': 7, 'BE': 8, 'AP': 9, 'CM': 10, 'CZ': 11, 'DN': 12, 'CU': 13, 'Dx': 14, 'Bb': 15, 'Er': 16, 'Bd': 17, 'DH': 18, 'Dc': 19, 'Ed': 20, 'Jt': 21, 'GD': 22, 'IK': 23, 'GJ': 24, 'Jc': 25, 'Hj': 26, 'Fd': 27, 'Gx': 28, 'FI': 29, 'Fo': 30, 'JX': 31, 'Eg': 32, 'Kq': 33, 'Nk': 34, 'BA': 35, 'Gb': 36, 'MC': 37, 'LE': 38, 'Mx': 39, 'KZ': 40, 'MO': 41, 'Os': 42, 'KR': 43, 'NV': 44, 'MV': 45, 'Ob': 46, 'Nf': 47, 'PQ': 48, 
                'Jf': 49, 'Qh': 50, 'Rm': 51, 'RP': 52, 'QM': 53, 'To': 54, 'Mf': 55, 'RL': 56, 'OR': 57, 'Sc': 58, 'Ml': 59, 'TZ': 60, 'Qo': 61, 'eg': 62, 'Ps': 63, 'Wv': 64, 'Uu': 65, 'Uk': 66, 'SB': 67, 'RG': 68, 'PA': 69, 'YC': 70, 'Xh': 71, 'PZ': 72, 'Yb': 73, 'ZR': 74, 'Ry': 75, 'Qb': 76, 'Zc': 77, 'Vf': 78, 'ZS': 79, 'TR': 80, 'XI': 81, 'Xi': 82, 'Ye': 83, 'UO': 84, 'aF': 85, 'WE': 86, 'Zq': 87, 'ac': 88, 'aM': 89, 'dE': 90, 'av': 91, 'aO': 92, 'Vx': 93, 'Wc': 94, 'aP': 95, 'eG': 96, 
                'bJ': 97, 'ck': 98, 'be': 99, 'bF': 100, 'dN': 101, 'cA': 102, 'dO': 103, 'dP': 104, 'cp': 105, 'cW': 106, 'dh': 107, 'cG': 108, 'eQ': 109, 'ih': 110, 'jp': 111, 'dZ': 112, 'eb': 113, 'hh': 114, 'ek': 115, 'hL': 116, 'dB': 117, 'ex': 118, 'fO': 119, 'hp': 120, 'je': 121, 'dQ': 122, 'gM': 123, 'iT': 124, 'kL': 125, 'fh': 126, 'hT': 127, 'kC': 128, 'jV': 129, 'gJ': 130, 'kw': 131, 'xP': 132, 'll': 133, 'kW': 134, 'kE': 135, 'kK': 136, 'ke': 137, 'oG': 138, 'lx': 139, 'qJ': 140, 'mm': 141, 'jS': 142, 'lL': 143, 'mc': 144, 
                'nh': 145, 'od': 146, 'oH': 147, 'oC': 148, 'qK': 149, 'qX': 150, 'lF': 151, 'ps': 152, 'on': 153, 'pa': 154, 'qA': 155, 'qo': 156, 'kr': 157, 'mb': 158, 'qP': 159, 'kU': 160, 'qw': 161, 'sn': 162, 'sD': 163, 'nX': 164, 'us': 165, 'tP': 166, 'sV': 167, 'oK': 168, 'qv': 169, 'ut': 170, 'rp': 171, 'ri': 172, 'rZ': 173, 'tv': 174, 'ud': 175, 'sY': 176, 'uy': 177, 'uS': 178, 'tM': 179, 'su': 180, 'vy': 181, 'uJ': 182, 'xy': 183, 'vK': 184, 'zU': 185, 'wy': 186, 'yN': 187, 'wu': 188, 'ux': 189, 'yY': 190, 'vq': 191, 'yc': 192}
    ord_maps = [ord0_map, ord1_map, ord2_map, ord3_map, ord4_map, ord5_map]
    for i, feature in enumerate(ordinal_features):
        temp_data[feature] = temp_data[feature].map(ord_maps[i])
    
    #day & month
    month_enc_map = {2: 1, 8: 6, 1: 2, 4: 4, 10: 9, 3: 3, 7: 7, 9: 8, 12: 10, 11: 11, 5: 5, 6: 12}
    day_enc_map = {2: 4, 7: 7, 5: 2, 4: 1, 3: 3, 1: 6, 6: 5}
    temp_data['month'] = temp_data['month'].map(month_enc_map)
    temp_data['day'] = temp_data['day'].map(day_enc_map)
    
    scaler = MinMaxScaler()
    temp_data['ord_3'] = scaler.fit_transform(np.array(temp_data['ord_3']).reshape(-1, 1))
    temp_data['ord_4'] = scaler.fit_transform(np.array(temp_data['ord_4']).reshape(-1, 1))
    temp_data['ord_5'] = scaler.fit_transform(np.array(temp_data['ord_5']).reshape(-1, 1))
    
    # move y column to the rightmost
    if is_train:
        x_cols = [col for col in temp_data.columns if col != 'target']
        temp_data = temp_data[x_cols + ['target']]
    
    return temp_data

In [17]:
preprocessed_train_data = preprocess_data(train_data, True)
preprocessed_test_data = preprocess_data(test_data, False)

In [20]:
preprocessed_test_data

,bin_0,bin_1,bin_2,bin_3,bin_4,ord_0,ord_1,ord_2,ord_3,ord_4,...,nom_3_Canada,nom_3_China,nom_3_Costa Rica,nom_3_Finland,nom_3_India,nom_3_Russia,nom_4_Bassoon,nom_4_Oboe,nom_4_Piano,nom_4_Theremin
0,0,0,1,1,1,2,1,3,0.642857,0.60,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0,0,1,0,1,4,6,0.857143,0.00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,0,1,0,1,2,3,1,0.000000,0.24,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,0,1,1,1,1,2,6,0.071429,0.64,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,1,1,0,0,3,5,6,0.857143,0.92,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0,0,0,0,0,1,1,6,0.642857,0.00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
199996,1,0,0,0,1,1,2,6,0.357143,0.72,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
199997,0,1,1,1,1,1,1,5,0.428571,0.84,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
199998,1,0,0,1,1,2,5,5,0.428571,0.88,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
preprocessed_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 39 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   bin_0             300000 non-null  int64  
 1   bin_1             300000 non-null  int64  
 2   bin_2             300000 non-null  int64  
 3   bin_3             300000 non-null  int64  
 4   bin_4             300000 non-null  int64  
 5   ord_0             300000 non-null  int64  
 6   ord_1             300000 non-null  int64  
 7   ord_2             300000 non-null  int64  
 8   ord_3             300000 non-null  float64
 9   ord_4             300000 non-null  float64
 10  ord_5             300000 non-null  float64
 11  day               300000 non-null  int64  
 12  month             300000 non-null  int64  
 13  nom_0_Blue        300000 non-null  float64
 14  nom_0_Green       300000 non-null  float64
 15  nom_0_Red         300000 non-null  float64
 16  nom_1_Circle      30

In [21]:
preprocessed_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 38 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   bin_0             200000 non-null  int64  
 1   bin_1             200000 non-null  int64  
 2   bin_2             200000 non-null  int64  
 3   bin_3             200000 non-null  int64  
 4   bin_4             200000 non-null  int64  
 5   ord_0             200000 non-null  int64  
 6   ord_1             200000 non-null  int64  
 7   ord_2             200000 non-null  int64  
 8   ord_3             200000 non-null  float64
 9   ord_4             200000 non-null  float64
 10  ord_5             200000 non-null  float64
 11  day               200000 non-null  int64  
 12  month             200000 non-null  int64  
 13  nom_0_Blue        200000 non-null  float64
 14  nom_0_Green       200000 non-null  float64
 15  nom_0_Red         200000 non-null  float64
 16  nom_1_Circle      20

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [29]:
x = preprocessed_train_data.drop('target', axis=1)
y = preprocessed_train_data.target

In [30]:
x.shape, y.shape

((300000, 38), (300000,))

In [31]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.25, stratify = y)

In [33]:
model = RandomForestClassifier(n_estimators=200,max_depth=15)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=15, n_estimators=200)

In [34]:
pred = model.predict(x_valid)

In [36]:
accuracy_score(y_valid, pred)

0.7362933333333334

In [37]:
f1_score(y_valid, pred)

0.3922315776534939

In [38]:
submit = model.predict(preprocessed_test_data)

In [39]:
submit

array([0, 0, 0, ..., 0, 0, 0])

In [40]:
submit_file = pd.read_csv('/kaggle/input/cat-in-the-dat/sample_submission.csv', index_col='id')

In [41]:
submit_file

,target
id,
300000,0.5
300001,0.5
300002,0.5
300003,0.5
300004,0.5
...,...
499995,0.5
499996,0.5
499997,0.5


In [42]:
submit_file.target = submit

In [47]:
submit_file.to_csv('sampleSubmission-v1.csv')